In [1]:
import sys
import numpy as np
import pandas as pd
import os
import joblib
import shap  # Import SHAP library
import seaborn as sns  # Import Seaborn for visualization
import matplotlib.pyplot as plt  # Import matplotlib for plotting
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, make_scorer
from sklearn.metrics import classification_report
from scipy import sparse as sp
from pathlib import Path

# Add project root to the Python path
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

from src.data_management.data_preprocessing_pipeline import DataPreprocessingPipeline


# Loading dataset
path = "/Users/jeancherizol/MADS/699/all_data"
path_models = "/Users/jeancherizol/MADS/699/SIADS/emergency-dept-optimization/models"
emergency_df = pd.read_sas(f"{path}/nhamcs14.sas7bdat")


target = 'WAITTIME_BINARY'
target_to_drop = ['WAITTIME','LOV_BINARY']
        
# Instantiate the data preprocessing pipeline
pipeline = DataPreprocessingPipeline(emergency_df=emergency_df,target=target,target_to_drop=target_to_drop,percent_train=0.80,percent_val=0.10,percent_test=0.10,path=path,stratify=False)

# Run the pipeline
pipeline.run()

X_train = pipeline.X_train
X_validation = pipeline.X_validation
X_test = pipeline.X_test

y_train = pipeline.y_train
y_validation = pipeline.y_validation
y_test = pipeline.y_test

#Save target variable 
file_name = "y_test_waittime_classification"
file_path = os.path.join(path, f"{file_name}.csv")
y_test.to_csv(file_path, index=False)
print(f"{file_name} dataset saved successfully")


X_train_preprocessed = pipeline.X_train_preprocessed
X_validation_preprocessed = pipeline.X_validation_preprocessed
X_test_preprocessed = pipeline.X_test_preprocessed


cleaned_emergency_df = pipeline.cleaned_emergency_df 
transformed_emergency_df = pipeline.transformed_emergency_df

cleaned_emergency_df['HOSPCODE'].value_counts()



1-Cleaning data...
Data cleaning completed
Size of Initial dataset:(23844, 1012)
Size of cleaned dataset:(21958, 408)

2-Applying feature engineering...
Feature engineering completed
Size of the dataset after feature engineering:(21958, 445)

3-Splitting data...
self.stratify: False
Splitting data completed

4-Loading data...
train_df size: (17566, 445)
X_train size: (17566, 444)
y_train size: (17566,)

validation_df size: (2196, 445)
X_validation size: (2196, 444)
y_validation size: (2196,)

test_df size: (2196, 445)
X_test size: (2196, 444)
y_test size: (2196,)
Loading data completed

5-Preprocessing data...
Preprocessing data completed.
Processor saved successfully
y_test_waittime_classification dataset saved successfully


HOSPCODE
2      266
149    213
244    162
77     157
189    156
      ... 
72      10
264      9
177      9
44       8
9        8
Name: count, Length: 239, dtype: int64

In [2]:
# Remove prefixes and convert to list in one step if not already a list
feature_names = [name.replace('num__', '').replace('cat__', '') for name in pipeline.feature_names]

# Top 60 most important features
# top_60_features = ['ARREMS', 'ARRTIME_IN_HOUR', 'DIFFEHRE', 'PATWT', 'DIAGSCRN', 'CSTRATM', 'EDINFO', 'NUMGIV', 'CPSUM', 'PHYSPRACTRIA', 'TOTDIAG', 'HOSPCODE', 'EMRED', 'IMMEDR', 'PULSE', 'WIRELESS', 'EMEDRES', 'BPSYS', 'EMSGE', 'RFV23D', 'RETREFFU', 'OTHPROV', 'MSA', 'EDPRIM', 'IVFLUIDS', 'FASTTRAK', 'TOTPROC', 'ESCRIPE', 'VMONTH', 'RFV33D', 'EREMINDE', 'EMSGER', 'BEDDATA', 'ADMDIV', 'MED1', 'DIAG3_-9', 'EGRAPHE', 'EBILLANYE', 'ESHAREPROVE2', 'EIDPTE', 'KIOSELCHK', 'CTHEAD', 'PAYMCAID', 'ECQMER', 'RFV2', 'AMBDIV', 'BPDIASD', 'ESHAREPROVE4', 'RFV1', 'PAYTYPER', 'URINECX', 'OTHERBLD', 'PROC', 'EPTRECE', 'RFV13D', 'ADVTRIAG', 'ESHAREPROVE7']
top_60_features = ['LOV', 'ARREMS', 'ARRTIME_IN_HOUR', 'DIFFEHRE', 'PATWT', 'DIAGSCRN', 'CSTRATM', 'EDINFO', 'NUMGIV', 'CPSUM', 'PHYSPRACTRIA', 'TOTDIAG', 'HOSPCODE', 'EMRED', 'IMMEDR', 'PULSE', 'WIRELESS', 'EMEDRES', 'BPSYS', 'EMSGE', 'RFV23D', 'RETREFFU', 'OTHPROV', 'MSA', 'EDPRIM', 'IVFLUIDS', 'FASTTRAK', 'TOTPROC', 'ESCRIPE', 'VMONTH', 'RFV33D', 'EREMINDE', 'EMSGER', 'BEDDATA', 'ADMDIV', 'MED1', 'DIAG3_-9', 'EGRAPHE', 'EBILLANYE', 'ESHAREPROVE2', 'EIDPTE', 'KIOSELCHK', 'CTHEAD', 'PAYMCAID', 'ECQMER', 'RFV2', 'AMBDIV', 'BPDIASD', 'ESHAREPROVE4', 'RFV1', 'PAYTYPER', 'URINECX', 'OTHERBLD', 'PROC', 'EPTRECE', 'RFV13D', 'ADVTRIAG', 'ESHAREPROVE7']
# # 

# Get indice of the top 60 most important features

feature_indices = [feature_names.index(feature) for feature in top_60_features]

# Get preprocessed data for the 60 most important features
# These will be used to train the model and evaluate its parameters
X_train_selected_features = X_train_preprocessed[:, feature_indices]
X_validation_selected_features = X_validation_preprocessed[:, feature_indices]
X_test_selected_features = X_test_preprocessed[:, feature_indices]

In [3]:
print('f',feature_indices )

f [2, 13, 382, 267, 381, 96, 379, 338, 208, 380, 361, 134, 261, 264, 31, 26, 366, 374, 28, 317, 40, 230, 226, 376, 337, 143, 362, 150, 287, 0, 41, 283, 318, 371, 352, 152, 3957, 299, 263, 329, 309, 357, 126, 18, 308, 35, 349, 216, 331, 34, 24, 119, 110, 135, 319, 39, 360, 334]


In [4]:

# Define a range of hyperparameters for CatBoostClassifier
param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [30, 50, 100],
    'l2_leaf_reg': [1, 3, 5]
}

# Initialize CatBoostClassifier with auto_class_weights set to 'Balanced'
catboost_model = CatBoostClassifier(random_state=42, verbose=0, auto_class_weights='Balanced')

# Configure GridSearchCV
grid_search = GridSearchCV(
    estimator=catboost_model,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    scoring=make_scorer(f1_score, average='weighted'),  # Using make_scorer to specify average method
    verbose=1
)

# Perform hyperparameter tuning
print("Starting hyperparameter tuning...")
grid_search.fit(X_train_selected_features, y_train)

# Retrieve and report the best hyperparameters
best_hyperparams = grid_search.best_params_
print(f"Best hyperparameters: {best_hyperparams}")

# Training the final model with best hyperparameters
best_model = grid_search.best_estimator_

# Evaluate on validation set
y_validation_pred = best_model.predict(X_validation_selected_features)
y_validation_proba = best_model.predict_proba(X_validation_selected_features)[:, 1]  # For ROC AUC

# Calculate metrics                                             
f1_validation = f1_score(y_validation, y_validation_pred, average='weighted')
roc_auc_validation = roc_auc_score(y_validation, y_validation_proba)
precision_validation = precision_score(y_validation, y_validation_pred, average='weighted')
recall_validation = recall_score(y_validation, y_validation_pred, average='weighted')

print(f"Validation Metrics:\nF1 (Weighted): {f1_validation:.2f}, ROC AUC: {roc_auc_validation:.2f}, Precision (Weighted): {precision_validation:.2f}, Recall (Weighted): {recall_validation:.2f}")



#Metrics for each class
# Generate a classification report
report = classification_report(y_validation, y_validation_pred, output_dict=True)

# Print out the classification report
print(classification_report(y_validation, y_validation_pred))

# For a more custom output, you can also access specific metrics from the report dictionary:
for label, metrics in report.items():
    if label not in ('accuracy', 'macro avg', 'weighted avg'):
        print(f"Class {label} - Precision: {metrics['precision']:.2f}, Recall: {metrics['recall']:.2f}, F1-score: {metrics['f1-score']:.2f}")


# Save the best model
model_filename = f"{path_models}/best_waittime_classification_catboost_model.joblib"
joblib.dump(best_model, model_filename)
print("best_waittime_classification_catboost_model saved")


Starting hyperparameter tuning...
Fitting 5 folds for each of 81 candidates, totalling 405 fits


Best hyperparameters: {'depth': 8, 'iterations': 100, 'l2_leaf_reg': 3, 'learning_rate': 0.1}
Validation Metrics:
F1 (Weighted): 0.73, ROC AUC: 0.82, Precision (Weighted): 0.76, Recall (Weighted): 0.72
              precision    recall  f1-score   support

           0       0.86      0.71      0.78      1509
           1       0.54      0.74      0.62       687

    accuracy                           0.72      2196
   macro avg       0.70      0.73      0.70      2196
weighted avg       0.76      0.72      0.73      2196

Class 0 - Precision: 0.86, Recall: 0.71, F1-score: 0.78
Class 1 - Precision: 0.54, Recall: 0.74, F1-score: 0.62
best_waittime_classification_catboost_model saved


In [5]:
# from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
# from sklearn.metrics import make_scorer, f1_score, classification_report
# from imblearn.over_sampling import SMOTE
# from imblearn.pipeline import Pipeline
# from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
# import joblib
# import numpy as np

# # Define the stratified K-fold for the imbalanced dataset
# cv_strategy = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# # Setup scoring function
# scoring = make_scorer(f1_score, average='weighted')

# # Function to create a pipeline and tune a given model
# def create_pipeline_and_tune(model_name, model, param_dist, X_train, y_train):
#     pipeline = Pipeline([
#         ('smote', SMOTE(random_state=42)),
#         (model_name, model)
#     ])
#     random_search = RandomizedSearchCV(
#         estimator=pipeline,
#         param_distributions=param_dist,
#         n_iter=10,
#         cv=cv_strategy,
#         scoring=scoring,
#         verbose=1,
#         random_state=42,
#         n_jobs=-1
#     )
#     random_search.fit(X_train, y_train)
#     return random_search.best_estimator_[model_name]

# # Define parameter distributions and models
# models_param_dist = {
#     'xgb': {
#         'xgb__n_estimators': [100, 200, 300],
#         'xgb__max_depth': [3, 5, 7],
#         'xgb__learning_rate': np.logspace(-3, -1, 3),
#     },
#     'catboost': {
#         'catboost__depth': [4, 6, 8],
#         'catboost__learning_rate': np.logspace(-3, -1, 3),
#         'catboost__iterations': [100, 200, 300],
#     },
#     'gb': {
#         'gb__n_estimators': [100, 200, 300],
#         'gb__learning_rate': np.logspace(-3, -1, 3),
#         'gb__max_depth': [3, 5, 7],
#     }
# }

# # Tuning models
# models = {
#     'xgb': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
#     'catboost': CatBoostClassifier(verbose=0, random_state=42, auto_class_weights='Balanced'),
#     'gb': GradientBoostingClassifier(random_state=42)
# }

# best_models = {}
# for name, model in models.items():
#     print(f"Tuning {name}...")
#     best_models[name] = create_pipeline_and_tune(name, model, models_param_dist[name], X_train, y_train)

# # Define and train the ensemble model
# ensemble_model = VotingClassifier(
#     estimators=[(name, best_models[name]) for name in best_models],
#     voting='soft'
# )

# final_pipeline = Pipeline([
#     ('smote', SMOTE(random_state=42)),
#     ('ensemble', ensemble_model)
# ])

# final_pipeline.fit(X_train, y_train)

# # Predict on validation set and evaluate
# y_validation_pred = final_pipeline.predict(X_validation)
# print(classification_report(y_validation, y_validation_pred))

# # Save the final model
# model_filename = f"{path_models}/best_waittime_classification_catboost_model.joblib"
# joblib.dump(final_pipeline, model_filename)
# print(f"{model_filename} saved")


# # # Save the final model
# # model_filename = f"{path_models}/best_waittime_classification_catboost_model.joblib"
# # joblib.dump(final_pipeline, model_filename)
# # print(f"{model_filename} saved")


In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, classification_report, precision_recall_fscore_support

# Assuming the best_model has been trained and is available
# Assuming X_test_selected_features, y_test, and HOSPCODEs are properly defined
HOSPCODEs_test = X_test['HOSPCODE'].values

# Prediction
y_test_pred = best_model.predict(X_test_selected_features)
# For binary classification tasks
y_test_proba = best_model.predict_proba(X_test_selected_features)[:, 1]

# Assuming HOSPCODEs_test contains the HOSPCODE for each record in X_test_selected_features
predictions_df = pd.DataFrame({
    'HOSPCODE': HOSPCODEs_test,
    'y_true': y_test,
    'y_pred': y_test_pred,
    'y_proba': y_test_proba
})

# Placeholder for aggregated results
results_per_hospcode = {}

valid_hospcode_list = []

for code, group in predictions_df.groupby('HOSPCODE'):
    report = classification_report(group['y_true'], group['y_pred'], output_dict=True, zero_division=0)
    metrics = precision_recall_fscore_support(group['y_true'], group['y_pred'], average=None, zero_division=0)
    
    # Unpack metrics for both classes
    precision_values, recall_values, f1_values, _ = metrics
    
    # Check if F1 scores for both classes are greater than 0.60 and precision and recall are less than 1
    if all(f1 >= 0.60 for f1 in f1_values) and all(p < 1 and r < 1 for p, r in zip(precision_values, recall_values)):
        valid_hospcode_list.append(code)
        
        # Displaying results for valid HOSPCODE
        print(f"HOSPCODE: {code}")
        print(f"Classification Report for {code}:")
        print(classification_report(group['y_true'], group['y_pred'], zero_division=0))
        print("\n")

print("List of HOSPCODEs meeting the criteria:")
print(valid_hospcode_list)

HOSPCODE: 31
Classification Report for 31:
              precision    recall  f1-score   support

           0       0.75      0.75      0.75         4
           1       0.67      0.67      0.67         3

    accuracy                           0.71         7
   macro avg       0.71      0.71      0.71         7
weighted avg       0.71      0.71      0.71         7



HOSPCODE: 91
Classification Report for 91:
              precision    recall  f1-score   support

           0       0.80      0.80      0.80         5
           1       0.67      0.67      0.67         3

    accuracy                           0.75         8
   macro avg       0.73      0.73      0.73         8
weighted avg       0.75      0.75      0.75         8



HOSPCODE: 101
Classification Report for 101:
              precision    recall  f1-score   support

           0       0.75      0.75      0.75         8
           1       0.60      0.60      0.60         5

    accuracy                           0.69    